In [ ]:
from pprint import pprint
import calendar
import time

import sklearn.datasets
import sklearn.metrics
import autosklearn.regression

import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import config

In [ ]:
with open('./data/inv_dataset.p', 'rb') as file:
    a = pickle.load(file)

In [ ]:
X, Y = a['X'], a['Y']
Xn, Yn = a['Xn'], a['Yn']
X_min, X_max = a['X_min'], a['X_max']
Y_min, Y_max = a['Y_min'], a['Y_max']

X_learn, Y_learn = a['X_learn'], a['Y_learn']
X_train, Y_train = a['X_train'], a['Y_train']
X_valid, Y_valid = a['X_valid'], a['Y_valid']
X_test , Y_test  = a['X_test'] , a['Y_test']

Xn_learn, Yn_learn = a['Xn_learn'], a['Yn_learn']
Xn_train, Yn_train = a['Xn_train'], a['Yn_train']
Xn_valid, Yn_valid = a['Xn_valid'], a['Yn_valid']
Xn_test , Yn_test  = a['Xn_test'] , a['Yn_test']

In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=1200,
    per_run_time_limit=60,
    tmp_folder=f"./temp/{int(calendar.timegm(time.gmtime()))}",
)
automl.fit(Xn_learn.numpy(), Yn_learn.numpy())

In [ ]:
Yn_learn_automl = automl.predict(Xn_learn.numpy())
Yn_test_automl  = automl.predict(Xn_test.numpy())
Y_learn_automl  = config.Normalization(Yn_learn_automl, Y_min.numpy(), Y_max.numpy(), inverse=True)
Y_test_automl   = config.Normalization(Yn_test_automl, Y_min.numpy(), Y_max.numpy(), inverse=True)

In [ ]:
print("Train R2 score:", sklearn.metrics.mean_squared_error(Y_learn.numpy(), Y_learn_automl))
print("Test R2 score:", sklearn.metrics.mean_squared_error(Y_test.numpy(), Y_test_automl))

In [ ]:
plt.scatter(Y_learn_automl, Y_learn.numpy(), label="Train samples", c="red", s=1)
plt.scatter(Y_test_automl, Y_test.numpy(), label="Test samples", c="blue", s=1)
plt.xlabel("Predicted value")
plt.ylabel("True value")
plt.legend()
plt.tight_layout()
plt.show()